In [1]:
from scripts.flight_data_processor import (
    fetch_paginated_data, # We might not use this now, but keep for consistency
    call_aviationstack,  # Make sure this is imported
    process_airlines_data, 
    process_airports_data, 
    process_routes_data, 
    process_flights_data
)
# Make sure to import the models
from scripts.models import Base, Flight, Airline, Airport, Route
import pandas as pd
import time
from datetime import datetime, timedelta 

# Create a fresh session if you don't have o
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine,  text

import os
from dotenv import load_dotenv

### Utilize historical flight data to predict the likelihood of delays for upcoming flights. Features can include departure and arrival airports, scheduled times, airline, and aircraft type. This project involves data preprocessing, model training, and deployment, offering a comprehensive MLOps experience.

- Try to pull to postgres from 2024-02-01 to 2024-04-31

In [6]:
# Load environment variables
load_dotenv()

# Database connection settings - adjust these as needed
DB_URL = os.getenv("DATABASE_URL")
API_KEY = os.getenv("AVIATIONSTACK_API_KEY")


# API Configuration
BASE_URL = "https://api.aviationstack.com/v1/"


In [3]:
# Create a session
engine = create_engine(DB_URL, echo=True) 
Session = sessionmaker(bind=engine)
session = Session()


## End-to-End Data Processing Test (Limited API Calls)

This section tests the complete workflow: Fetch -> Process -> Store.

**Crucially, this test makes exactly 3 API calls for EACH endpoint (Airlines, Airports, Routes, Flights) to precisely control API usage during testing.**

We use the lower-level `call_aviationstack` function for this control.

In [4]:
# --- Test Parameters ---
API_CALLS_PER_ENDPOINT = 3
RESULTS_PER_CALL = 10 # Number of records to fetch per API call
TEST_DATE = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d") # Use day before yesterday

print(f"--- Starting End-to-End Test ---")
print(f"Testing with {API_CALLS_PER_ENDPOINT} API calls per endpoint.")
print(f"Fetching {RESULTS_PER_CALL} results per call.")
print(f"Using TEST_DATE = {TEST_DATE} for flights.")

# Ensure session is active (recreate if needed)
# Note: process_*_data functions handle their own commits and potentially session closing.
# We might need to reopen the session if one closes it unexpectedly.
def get_session():
    if not session.is_active:
        print("Recreating database session...")
        NewSession = sessionmaker(bind=engine)
        return NewSession()
    return session

current_session = get_session()

try:
    # --- Helper Function for Looped API Calls & Processing ---
    def test_endpoint(endpoint_name, process_func, date=None):
        print(f"\n[{endpoint_name.upper()}] Testing Endpoint...")
        all_data_for_endpoint = []
        total_processed = 0
        
        for i in range(API_CALLS_PER_ENDPOINT):
            offset = i * RESULTS_PER_CALL
            print(f"-- {endpoint_name} Call {i+1}/{API_CALLS_PER_ENDPOINT} (Offset: {offset}) --")
            
            params = {"limit": RESULTS_PER_CALL, "offset": offset}
            if date:
                params["flight_date"] = date
                
            # Make the API call
            response_data = call_aviationstack(endpoint_name.lower(), params) # Use lower case for endpoint
            
            if response_data and "error" not in response_data:
                batch_data = response_data.get("data", [])
                print(f"API call successful. Fetched {len(batch_data)} records.")
                
                if batch_data:
                    all_data_for_endpoint.extend(batch_data)
                    # Process this batch
                    print("Processing batch...")
                    # Get a potentially fresh session if the previous process function closed it
                    sess = get_session() 
                    # process_func should handle its own commit based on its implementation
                    # If process_func returns count, capture it
                    count = process_func(batch_data, sess) 
                    if isinstance(count, int): # process_flights_data returns count
                         total_processed += count
                    else: # Other process functions might not return count
                         total_processed += len(batch_data) # Estimate processed
                    print("Batch processed.")
                else:
                    print("No more data found at this offset.")
                    # break # Optional: stop if API returns empty list early
            else:
                print(f"Error during API call: {response_data.get('error', 'Unknown error')}")
                # break # Optional: stop if there's an error

            # Delay between API calls
            print("Waiting before next call...")
            time.sleep(1.5) # Be respectful of API limits

        print(f"[{endpoint_name.upper()}] Finished testing. Fetched approx {len(all_data_for_endpoint)} records, processed approx {total_processed}.")
        return all_data_for_endpoint

    # --- Execute Tests for Each Endpoint ---
    # test_endpoint("Airlines", process_airlines_data)
    # test_endpoint("Airports", process_airports_data)
    test_endpoint("Routes", process_routes_data)
    test_endpoint("Flights", process_flights_data, date=TEST_DATE)

except Exception as e:
    print(f"\n--- TEST FAILED ---")
    print(f"An error occurred during the test: {str(e)}")
    # Rollback might not be needed if process_func handles its transactions,
    # but doesn't hurt if session is still active.
    if current_session.is_active:
         current_session.rollback()
else:
     print(f"\n--- TEST COMPLETED (Limited API Calls) ---")
finally:
    # Ensure the session is closed at the very end
    if 'current_session' in locals() and current_session.is_active:
        current_session.close()
        print("Final database session closed.")
    else:
        print("Session was already closed.")

--- Starting End-to-End Test ---
Testing with 3 API calls per endpoint.
Fetching 10 results per call.
Using TEST_DATE = 2025-05-03 for flights.

[AIRLINES] Testing Endpoint...
-- Airlines Call 1/3 (Offset: 0) --
API call successful. Fetched 10 records.
Processing batch...
2025-05-05 16:40:03,293 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2025-05-05 16:40:03,293 - sqlalchemy.engine.Engine - INFO - select pg_catalog.version()


2025-05-05 16:40:03,294 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-05-05 16:40:03,294 - sqlalchemy.engine.Engine - INFO - [raw sql] {}


2025-05-05 16:40:03,297 INFO sqlalchemy.engine.Engine select current_schema()


...

Error during API call: 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=0&access_key=326d0f100b4be2a5f900ec713da1bafb
Waiting before next call...
-- Routes Call 2/3 (Offset: 10) --


2025-05-05 16:40:28,173 - scripts.flight_data_processor - ERROR - Request failed (attempt 1/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=10&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:28,174 - scripts.flight_data_processor - INFO - Retrying in 2 seconds...
2025-05-05 16:40:31,118 - scripts.flight_data_processor - ERROR - Request failed (attempt 2/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=10&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:31,119 - scripts.flight_data_processor - INFO - Retrying in 4 seconds...
2025-05-05 16:40:36,035 - scripts.flight_data_processor - ERROR - Request failed (attempt 3/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=10&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:36,036 - scripts.flight_data_processor - ERROR - Max retries exceeded


Error during API call: 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=10&access_key=326d0f100b4be2a5f900ec713da1bafb
Waiting before next call...
-- Routes Call 3/3 (Offset: 20) --


2025-05-05 16:40:38,492 - scripts.flight_data_processor - ERROR - Request failed (attempt 1/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=20&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:38,493 - scripts.flight_data_processor - INFO - Retrying in 2 seconds...
2025-05-05 16:40:41,406 - scripts.flight_data_processor - ERROR - Request failed (attempt 2/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=20&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:41,407 - scripts.flight_data_processor - INFO - Retrying in 4 seconds...
2025-05-05 16:40:46,283 - scripts.flight_data_processor - ERROR - Request failed (attempt 3/3): 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=20&access_key=326d0f100b4be2a5f900ec713da1bafb
2025-05-05 16:40:46,284 - scripts.flight_data_processor - ERROR - Max retries exceeded


Error during API call: 403 Client Error: Forbidden for url: https://api.aviationstack.com/v1/routes?limit=10&offset=20&access_key=326d0f100b4be2a5f900ec713da1bafb
Waiting before next call...
[ROUTES] Finished testing. Fetched approx 0 records, processed approx 0.

--- TEST COMPLETED (Limited API Calls) ---
Final database session closed.


## Verification

Let's query the database to see if data was inserted/updated by the limited test calls. The counts might be small due to the limited calls.

In [9]:
# --- Verification ---
print("--- Verifying Data in Database ---")

# Recreate engine if needed for query
engine = create_engine(DB_URL) 

try:
    print("\n-- Airlines Table (Sample) --")
    #airlines_df = pd.read_sql("SELECT iata_code, airline_name, country_name FROM airlines ORDER BY iata_code LIMIT 10", engine)
    airlines_df = pd.read_sql("SELECT * FROM airlines ORDER BY iata_code LIMIT 10", engine)
    display(airlines_df)

    print("\n-- Airports Table (Sample) --")
    airports_df = pd.read_sql("SELECT * FROM airports ORDER BY iata_code LIMIT 10", engine)
    display(airports_df)
    
    print("\n-- Routes Table (Sample) --")
    routes_df = pd.read_sql("SELECT airline_iata, flight_number, departure_iata, arrival_iata FROM routes LIMIT 10", engine)
    if not routes_df.empty:
        display(routes_df)
    else:
        print("No routes found in the table (might be expected with limited test data).")

    print(f"\n-- Flights Table (Sample)--")
    flights_df = pd.read_sql(f"SELECT flight_date, flight_status, airline_name, flight_iata, departure_iata, arrival_iata FROM flights LIMIT 10", engine)
    if not flights_df.empty:
        display(flights_df)
    else:
        print(f"No flights found in the table.")
        
    # Add counts
    print("\n-- Record Counts --")
    #for table in ['airlines', 'airports', 'routes', 'flights']:
    for table in ['airlines', 'airports']:
        count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0]['count']
        print(f"Total records in {table}: {count}")


except Exception as e:
    print(f"\nError during verification query: {str(e)}")

print("\n--- Verification Complete ---")

--- Verifying Data in Database ---

-- Airlines Table (Sample) --


,id,iata_code,airline_id,icao_code,iata_prefix_accounting,airline_name,callsign,country_name,country_iso2,date_founded,hub_code,fleet_size,fleet_average_age,status,type,raw_payload
0,4598187,02,737,CPB,None,Corporate Express,PENTA,Canada,CA,NaN,None,NaN,NaN,unknown,None,"{'id': '4598187', 'type': None, 'status': 'unk..."
1,4598188,0A,738,GNT,None,Amber Air,GINTA,Lithuania,LT,NaN,VNO,NaN,NaN,historical,scheduled,"{'id': '4598188', 'type': 'scheduled', 'status..."
2,4597731,0B,281,BMS,475,Blue Air,BLUE MESSENGER,Romania,RO,2004.0,OTP,15.0,23.6,active,scheduled,"{'id': '4597731', 'type': 'scheduled', 'status..."
3,4598189,0C*,739,None,None,IBL Aviation Ltd,None,Mauritius,MU,NaN,None,NaN,NaN,historical,None,"{'id': '4598189', 'type': None, 'status': 'his..."
4,4598190,0J,740,PJZ,254,Premium Jet,PRIMEJET,Switzerland,CH,NaN,ZRH,NaN,NaN,active,private,"{'id': '4598190', 'type': 'private', 'status':..."
5,4598191,0J*,741,JCS,None,Jetclub,JETCLUB,Switzerland,CH,NaN,ZRH,NaN,NaN,historical,"scheduled,charter","{'id': '4598191', 'type': 'scheduled,charter',..."
6,4598192,0K,742,KRT,None,Aircompany Kokshetau,KOKTA,Kazakhstan,KZ,NaN,KOV,NaN,NaN,historical,scheduled,"{'id': '4598192', 'type': 'scheduled', 'status..."
7,4598193,0P,743,PPC,840,Palau Asia Pacific Air,None,Palau,PW,NaN,ROR,NaN,NaN,historical,scheduled,"{'id': '4598193', 'type': 'scheduled', 'status..."
8,4598194,0Q,744,HYD,None,Hydro - Quebec,None,None,None,NaN,None,NaN,NaN,active,None,"{'id': '4598194', 'type': None, 'status': 'act..."
9,4598028,0Q*,578,HYD,None,Hydro-Quebec,HYDRO,Canada,CA,NaN,YUL,3.0,16.1,active,government,"{'id': '4598028', 'type': 'government', 'statu..."



-- Airports Table (Sample) --


,iata_code,airport_name,icao_code,latitude,longitude,geoname_id,city_iata_code,country_name,country_iso2,timezone,gmt,phone_number,raw_payload
0,AAA,Anaa,NTGA,-17.050000,-145.416670,6947726,AAA,French Polynesia,PF,Pacific/Tahiti,-10,None,"{'id': '3610983', 'gmt': '-10', 'latitude': '-..."
1,AAB,Arrabury,YARY,-26.700000,141.041670,7730796,AAB,Australia,AU,Australia/Brisbane,10,None,"{'id': '3610984', 'gmt': '10', 'latitude': '-2..."
2,AAC,El Arish International Airport,HEAR,31.133333,33.750000,6297289,AAC,Egypt,EG,Africa/Cairo,2,None,"{'id': '3610985', 'gmt': '2', 'latitude': '31...."
3,AAE,Les Salines,DABB,36.821392,7.811857,2570559,AAE,None,DZ,Africa/Algiers,1,None,"{'id': '3610986', 'gmt': '1', 'latitude': '36...."
4,AAF,Apalachicola Regional,KAAF,29.733334,-84.983330,4146153,AAF,United States,US,America/New_York,-5,None,"{'id': '3610987', 'gmt': '-5', 'latitude': '29..."
5,AAG,Arapoti,SSYA,-24.103611,-49.790000,3471795,AAG,Brazil,BR,America/Sao_Paulo,-3,None,"{'id': '3610988', 'gmt': '-3', 'latitude': '-2..."
6,AAH,Aachen/Merzbruck,EDKA,50.750000,6.133333,3207669,AAH,Germany,DE,Europe/Berlin,1,None,"{'id': '3610989', 'gmt': '1', 'latitude': '50...."
7,AAI,Arraias,SWRA,-12.916667,-46.933334,7668483,AAI,Brazil,BR,America/Araguaina,-3,None,"{'id': '3610990', 'gmt': '-3', 'latitude': '-1..."
8,AAK,Aranuka,NGUK,0.166667,173.583330,7521791,AAK,Kiribati,KI,Pacific/Tarawa,12,None,"{'id': '3610992', 'gmt': '12', 'latitude': '0...."
9,AAL,Aalborg,EKYT,57.086550,9.872241,2624887,AAL,Denmark,DK,Europe/Copenhagen,1,None,"{'id': '3610993', 'gmt': '1', 'latitude': '57...."



-- Routes Table (Sample) --


,airline_iata,flight_number,departure_iata,arrival_iata
0,0B,101,OTP,TRN
1,0B,1015,OTP,FCO
2,0B,1016,FCO,OTP
3,0B,102,TRN,OTP
4,0B,103,OTP,MAD
5,0B,1038,LCA,OTP
6,0B,104,MAD,OTP
7,0B,105,OTP,BVA
8,0B,106,BVA,OTP
9,0B,107,OTP,BCN



-- Flights Table (Sample)--
No flights found in the table.

-- Record Counts --
Total records in airlines: 13143
Total records in airports: 6710

--- Verification Complete ---
